In [ ]:
#Где находятся пики относительно геномной разметки?
#Как связаны между собой пики, имеющие отношение к одному гену?
#Как связаны между собой корреляции пиков, имеющих отношение к одному гену? Все ли пики в одном гене коррелируют с конкретной 
#РНК? Почему у корреляции с пиками в разных генах может быть разный знак?
#Есть ли гены без пиков?
#Попробовать все же бинарную корреляцию?
#Как пики соотносятся с фантомовскими промоторами?

In [38]:
import pandas as pd
import re
import mygene

In [2]:
#Фантомовские старты транскрипции
##ColumnVariables[00Annotation]=CAGE peak id
##ColumnVariables[short_description]=short form of the description below. 
#Common descriptions in the long descriptions has been omited
##ColumnVariables[description]=description of the CAGE peak
##ColumnVariables[association_with_transcript]=transcript which 5end is the nearest to the the CAGE peak
##ColumnVariables[entrezgene_id]=entrezgene (genes) id associated with the transcript
##ColumnVariables[hgnc_id]=hgnc (gene symbol) id associated with the transcript
##ColumnVariables[uniprot_id]=uniprot (protein) id associated with the transcript
fantom_promoters = pd.read_csv("../fantom6/hg19.cage_peak_phase1and2combined_ann.txt", sep="\t", skiprows=7)

In [3]:
fantom_promoters.head()

,00Annotation,short_description,description,association_with_transcript,entrezgene_id,hgnc_id,uniprot_id
0,"chr10:100013403..100013414,-","p@chr10:100013403..100013414,-","CAGE_peak_at_chr10:100013403..100013414,-",NaN,NaN,NaN,NaN
1,"chr10:100027943..100027958,-",p1@LOXL4,CAGE_peak_1_at_LOXL4_5end,"48bp_to_ENST00000260702,NM_032211,uc001kpa.1_5end",entrezgene:84171,HGNC:17171,uniprot:Q96JB6
2,"chr10:100076685..100076699,+","p@chr10:100076685..100076699,+","CAGE_peak_at_chr10:100076685..100076699,+",NaN,NaN,NaN,NaN
3,"chr10:100150910..100150935,-","p@chr10:100150910..100150935,-","CAGE_peak_at_chr10:100150910..100150935,-",NaN,NaN,NaN,NaN
4,"chr10:100150951..100150962,-","p@chr10:100150951..100150962,-","CAGE_peak_at_chr10:100150951..100150962,-",NaN,NaN,NaN,NaN


In [4]:
fantom_promoters.shape

(201802, 7)

In [11]:
with_gene = fantom_promoters[fantom_promoters.hgnc_id.notnull()]

In [27]:
parse_ann = with_gene['00Annotation'].str.extract('(.+):(\d+)..(\d+).(-|\+)', expand=True)

In [30]:
parse_ann = parse_ann.rename(columns={0: "chrom", 1: "start", 2: "end", 3:"strand"})

In [33]:
with_gene = pd.concat([parse_ann, with_gene], axis=1).drop(['00Annotation'], axis=1)

In [36]:
with_gene[['chrom', 'start', 'end', 'strand', 'hgnc_id']].to_csv("../fantom6/tss_with_hgnc.tsv", sep="\t", index=None)

In [ ]:
#=========================================================================

In [81]:
promts = pd.read_csv("../fantom6/tss_with_hgnc.tsv", sep="\t")

In [72]:
promts = pd.concat([promts, promts.hgnc_id.str.split(',', expand=True)], axis=1).drop(['hgnc_id'], axis=1)

In [73]:
promts = pd.melt(promts, id_vars =['chrom','start','end','strand'], value_vars =[0,1,2,3,4,5,6,7,8,9,10,11,12,13])

In [74]:
promts = promts[promts.value.notnull()]

In [78]:
promts = promts[['chrom','start','end','strand','value']].rename(columns={'value': 'hgnc_id'}).to_csv("../fantom6/tss_with_hgnc.tsv", sep="\t", index=None)

In [140]:
promts.shape

(88521, 5)

In [ ]:
#===========================================================================

In [125]:
hgnc_ids = promts.hgnc_id.unique()

In [126]:
hgnc_ids

array(['HGNC:17171', 'HGNC:23517', 'HGNC:5163', ..., 'HGNC:4790',
       'HGNC:4791', 'HGNC:4793'], dtype=object)

In [134]:
hgnc2ensembl = pd.read_csv("../fantom6/hgnc_to_ensembl.tsv", sep="\t")

In [133]:
hgnc2ensembl[~hgnc2ensembl['Ensembl gene ID'].isna()].to_csv("../fantom6/hgnc_to_ensembl.tsv", sep="\t", index=None)

In [138]:
tmp = hgnc2ensembl[hgnc2ensembl['HGNC ID'].isin(hgnc_ids)].rename(columns={'HGNC ID': 'hgnc_id', 'Ensembl gene ID': 'ensembl_id'})

In [141]:
t = pd.merge(promts, tmp, on='hgnc_id', how='left')

In [144]:
t[~t['ensembl_id'].isna()].to_csv("../fantom6/tss_with_hgnc.tsv", sep="\t", index=None)

In [ ]:
#Если пики разного знака, то они в разных промоторах